In [1]:
import pyxatu

xatu = pyxatu.PyXatu()

/Users/francis/.asdf/installs/python/3.12.4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-07 09:28:02,421 - INFO - Clickhouse configs set
2024-10-07 09:28:02,422 - INFO - Clickhouse URL: https://clickhouse.analytics.production.platform.ethpandaops.io, User: base


Config Path:  /Users/francis/.pyxatu_config.json
Table canonical_beacon_elaborated_attestation not found.
Table beacon_api_eth_v1_events_attestation not found.
Table beacon_api_eth_v2_beacon_block not found.
Table beacon_api_eth_v1_events_blob_sidecar not found.
Table canonical_beacon_blob_sidecar not found.
Table beacon_api_eth_v1_events_block not found.
Table canonical_beacon_block not found.
Table canonical_execution_transaction not found.
Table canonical_beacon_block_execution_transaction not found.
Table canonical_beacon_block not found.
Table canonical_beacon_proposer_duty not found.
Table beacon_api_eth_v1_events_chain_reorg not found.
Table canonical_beacon_block not found.
Table canonical_beacon_block_execution_transaction not found.
Table canonical_beacon_block_withdrawal not found.


In [2]:
slots_per_day = 24 * 60 * 60 // 12
days = 28
# Oct-06-2024 12:20:23 AM UTC
end_slot = 10112500       
# Oct-05-2024 12:20:23 AM UTC, for some reason, the last several hundred roles are not populated with data
# so deduct one day worth of slot from the end_slot
real_end_slot = end_slot - slots_per_day
# Sep-21-2024 12:20:23 AM UTC, 15 days from end slot
start_slot = end_slot - (days + 1) * slots_per_day

df = xatu.get_block_size(
    slot = [start_slot, end_slot],
    orderby = "slot"
)

df = df[:days * slots_per_day]
df

2024-10-07 09:28:47,955 - INFO - Executing query: SELECT DISTINCT slot,block_total_bytes_compressed,block_total_bytes,execution_payload_blob_gas_used,execution_payload_transactions_total_bytes,execution_payload_transactions_total_bytes_compressed FROM canonical_beacon_block FINAL WHERE slot_start_date_time >= '2024-09-07 00:20:23' AND slot_start_date_time < '2024-10-06 00:20:23' AND slot >= 9903700 AND slot < 10112500 AND meta_network_name = 'mainnet'


Table canonical_beacon_block not found.


2024-10-07 09:28:50,152 - INFO - Query executed in 2.20 seconds


,slot,block_total_bytes_compressed,block_total_bytes,execution_payload_transactions_total_bytes,execution_payload_transactions_total_bytes_compressed,blobs
0,10031814,89339,205901,54948,29710,3
1,10031817,72466,174878,60983,30382,0
2,10031818,71479,176642,61952,29369,6
3,10031819,80762,185877,66413,34740,0
4,10031820,86691,220482,62349,29681,0
...,...,...,...,...,...,...
201595,9980706,84416,208029,56180,27982,1
201596,9980708,78331,181959,43191,24420,0
201597,9980709,103024,249033,76347,38616,6
201598,9980713,67188,166914,57846,27424,0


In [4]:
import pandas as pd
import numpy as np

# Calculate percentiles
percentiles = df['block_total_bytes'].quantile([0.50, 0.90, 0.95, 0.99, 0.999, 0.9999])

# Calculate average
average = df['block_total_bytes'].mean()

# Create a result DataFrame
result = pd.DataFrame({
    'Metric': ['Average', 'p50', 'p90', 'p95', 'p99', 'p999', 'p9999'],
    'Value (bytes)': [
        average,
        percentiles[0.50],
        percentiles[0.90],
        percentiles[0.95],
        percentiles[0.99],
        percentiles[0.999],
        percentiles[0.9999]
    ]
})

# Convert bytes to KB
result['Value (KB)'] = result['Value (bytes)'] / 1024

# Format the 'Value (bytes)' column to show whole numbers
result['Value (bytes)'] = result['Value (bytes)'].round(0).astype(int)

# Format the 'Value (KB)' column to show 2 decimal places
result['Value (KB)'] = result['Value (KB)'].round(2)

# Display the result
print(result.to_string(index=False))

 Metric  Value (bytes)  Value (KB)
Average         221837      216.64
    p50         214216      209.19
    p90         293279      286.41
    p95         333515      325.70
    p99         458935      448.18
   p999         628405      613.68
  p9999         854344      834.32


In [5]:
import pandas as pd
import numpy as np

# Calculate percentiles
percentiles = df['block_total_bytes_compressed'].quantile([0.50, 0.90, 0.95, 0.99, 0.999, 0.9999])

# Calculate average
average = df['block_total_bytes_compressed'].mean()

# Create a result DataFrame
result = pd.DataFrame({
    'Metric': ['Average', 'p50', 'p90', 'p95', 'p99', 'p999', 'p9999'],
    'Value (bytes)': [
        average,
        percentiles[0.50],
        percentiles[0.90],
        percentiles[0.95],
        percentiles[0.99],
        percentiles[0.999],
        percentiles[0.9999]
    ]
})

# Convert bytes to KB
result['Value (KB)'] = result['Value (bytes)'] / 1024

# Format the 'Value (bytes)' column to show whole numbers
result['Value (bytes)'] = result['Value (bytes)'].round(0).astype(int)

# Format the 'Value (KB)' column to show 2 decimal places
result['Value (KB)'] = result['Value (KB)'].round(2)

# Display the result
print(result.to_string(index=False))

 Metric  Value (bytes)  Value (KB)
Average          90116       88.00
    p50          86703       84.67
    p90         118401      115.63
    p95         134861      131.70
    p99         186793      182.42
   p999         305523      298.36
  p9999         387555      378.47
